In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
%matplotlib qt

Los archivos del R&S están formateados con las fechas y horas, vamos a sacar esa información de los nombres

In [2]:
# tiempos = [fecha_hora] sacada de los nombres de los archivos: (c/u a su vez en una carpeta conteniendo al csv y al wfm.csv correspondiente)
tiempos = list(filter(lambda x: x.startswith('2021'),os.listdir('datos_2Tonos')))
# ordenamos esta lista con la cronologia real
t0 = tiempos[0]
def tiempoReal(t):
    """YYYY_..._SS --> tiempo real (~7 hs menos (ver cuaderno L7)), en formato pandas"""
    fechaHora = t[:10] + '_' + t[-6:]
    tiempo = pd.to_datetime(fechaHora,format='%Y-%m-%d_%H%M%S')
    return tiempo - pd.Timedelta('07:48:00') #ver hoja 16 cuaderno L7


In [3]:
tiempos[0]

'2021-12-22_1_231924'

In [57]:
str(tiempoReal(t0))

tiempos2 = 

['2021-12-22_26_233158',
 '2021-12-22_27_233346',
 '2021-12-22_28_233351',
 '2021-12-22_29_234236',
 '2021-12-22_30_234243',
 '2021-12-22_31_234249',
 '2021-12-22_32_234342',
 '2021-12-22_33_234346',
 '2021-12-23_0_000334',
 '2021-12-23_1_000341',
 '2021-12-23_2_000352',
 '2021-12-23_3_000358',
 '2021-12-23_4_000457',
 '2021-12-23_5_000522',
 '2021-12-23_6_000526',
 '2021-12-23_7_001025',
 '2021-12-23_8_001030']

In [44]:
for i in range(len(tiempos)): print(tiempoReal(tiempos[i]))

2021-12-22 15:43:58
2021-12-22 15:45:46
2021-12-22 15:45:51
2021-12-22 15:54:36
2021-12-22 15:54:43
2021-12-22 15:54:49
2021-12-22 15:55:42
2021-12-22 15:55:46
2021-12-22 16:15:34
2021-12-22 16:15:41
2021-12-22 16:15:52
2021-12-22 16:15:58
2021-12-22 16:16:57
2021-12-22 16:17:22
2021-12-22 16:17:26
2021-12-22 16:22:25
2021-12-22 16:22:30


In [46]:
def params(csv):
    """
    Lee los datos relevantes del archivo csv de parametros, i.e.
    los que terminan en .csv y no los .wfm.csv
    Devuelve un diccionario dicc[parametro]=valor
    Los valores sirven para graficar (e.g. escalas)
    """
    with open(csv) as file:
        lines = file.readlines()
    parametros = [
        'Resolution',
        'TimeScale',
        'Source'
    ]
    dicc = {}
    for p in parametros:
        for x in lines:
            if x.startswith(p+':'):
                startIndex = x.find(':')
                endIndex = x.rfind(':')
                value = x[startIndex+1:endIndex]
                dicc[p] = value
    return dicc

In [48]:
def nombre_csv(t):
    n1 = 'datos_2Tonos/'
    n2 = '/RefCurve_'
    n3 = '.csv'
    return n1 + t + n2 + t + n3

def nombre_wfmcsv(t):
    n1 = 'datos_2Tonos/'
    n2 = '/RefCurve_'
    n3 = '.Wfm.csv'
    return n1 + t + n2 + t + n3

In [49]:
csv = nombre_wfmcsv(tiempos[0])
print(csv)
data = pd.read_csv(csv)
'datos_2Tonos\2021-12-22_1_231924\RefCurve_2021-12-22_1_231924.Wfm.csv'

datos_2Tonos/2021-12-22_26_233158/RefCurve_2021-12-22_26_233158.Wfm.csv


'datos_2Tonos\x821-12-22_1_231924\\RefCurve_2021-12-22_1_231924.Wfm.csv'

In [50]:
data.head()

,0.09152174
0,0.091522
1,0.091522
2,0.091522
3,0.091522
4,0.091522


In [69]:
def csv_plot_RS(csv,plotear=True):
    """Agarra un archivo csv (del osciloscopio Rohde & Schwarz)
     y lo grafica.\n
     La variable csv es un txt con el nombre del archivo csv\n
     Cuando sale del R&S, el nombre tiene la estructura:\n
     RefCurve_YYYY-MM-DD_N_HHMMSS.Wfm.csv\n
     Donde N es un numero que ordena las wfm de ese dia
     """
    data = pd.read_csv(csv,sep=';',names=['x'])
    i = csv.rfind('Curve') + len('Curve') + 1
    j = csv.find('.Wfm')
    text = csv[i:j] # text = 'YYYY-MM-DD_HHMMSS'
    x = data['x']
    if plotear==True:
        plt.plot(x)
        plt.grid()
        plt.title(text)
        plt.show()
    return x

def fft_plot(csv,T,plotearWfm=False):
    """Agarra el path del csv, le hace FFT, lo grafica"""
    plt.cla()
    if plotearWfm==True:
        plt.subplot(2,1,1)
        y = csv_plot_RS(csv,plotear=plotearWfm)
        U = np.fft.fft(y)
        #T = 4.16666e-005 # este tiempo en segundos se saca de los datos del R&S
        t = T*np.arange(len(y))
        freq = np.fft.fftfreq(t.shape[-1],d=T)
        N = len(freq)
        U = U[:N//2]
        freqs = 1e-3*freq[:N//2] #frecuencias positivas (en kHz)
        Uy = np.sqrt(U.real**2+U.imag**2) #valores absolutos
        #plt.plot(freqs, Ue,label='FFT($e_q$)',zorder=0,color="y")
        plt.subplot(2,1,2)
    plt.plot(freqs,Uy,label='FFT($y$)',zorder=1,color="k",marker='o')
    plt.title("ffty")
    plt.yscale('log')
    #plt.xscale()
    plt.grid()
    #plt.xlim(10**-3,10**4)
    plt.xlabel('Frecuencia [kHz]')
    plt.ylabel('Magnitud [dB]')
    plt.legend()
    plt.show()

In [52]:
def decimador(pulsos,N):
    """Agarra la lista pulsos, que es una
       lista de 1's y 0's (ints) y hace un promedio 
       temporal tomando chunks de N pulsos.
       Devuelve una lista de largo 'LargoOriginal/N'
       conteniendo los promedios de cada chunk de
       N pulsos. La ultima sublista es el resto, no
       necesariamente tenga longitud N.
       Se puede usar para decimar listas de floats tambien
       (i.e. no solo 0's y 1's)
       """
    chunks = [pulsos[i:i+N] for i in range(0,len(pulsos),N)]
    promedios = []
    for c in chunks:
        promedios.append(np.mean(c))
    return promedios

In [70]:
def plotear(t0):
    csv = nombre_wfmcsv(t0)
    csvParam = nombre_csv(t0)
    pars = params(csvParam)
    source = pars['Source']
    if source.startswith('Ch'):
        T = float(pars['TimeScale'])
        fft_plot(csv,T,plotearWfm=True)

In [72]:
%matplotlib qt
t0 = tiempos[4]
plotear(t0)